In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [13]:
stats = pd.read_csv(r'nbaStats.csv')
classNum = 9


stats.dropna(inplace=True)
pos = stats.pop('Pos')
stats_train, stats_test, pos_train, pos_test = train_test_split(stats, pos, test_size=0.2)
pos_vals = list(pos.unique())
pos_nums = [0, 1, 2, 3, 4, 5, 6, 7, 8]
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(pos_vals)

#Convert categorical df to numeric
pos_train = list(pos_train)
for i in range(len(pos_train)):
    pos_train[i] = pos_vals.index(str(pos_train[i]))
pos_train = pd.DataFrame(pos_train)

#Convert categorical df to numeric
pos_test = list(pos_test)
for i in range(len(pos_test)):
    pos_test[i] = pos_vals.index(str(pos_test[i]))
pos_test = pd.DataFrame(pos_test)


print(pos_test.shape)
print(stats_test.shape)
print(stats_train.columns)

['C', 'PF', 'SG', 'SF', 'PG', 'SF-SG', 'C-PF', 'PF-SF', 'SF-PF']
(112, 1)
(112, 29)
Index(['Rk', 'Player', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [17]:
CATEGORICAL_COLUMNS = ['Tm']
NUMERIC_COLUMNS = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = stats_train[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [22]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(stats_train, pos_train, num_epochs=10)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(stats_test, pos_test, num_epochs=1, shuffle=False)

In [10]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns, 
                                           n_classes=classNum)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])
print(result)

0.3482143
{'accuracy': 0.3482143, 'average_loss': 2.3517826, 'loss': 2.3036354, 'global_step': 140}


In [6]:
result = list(linear_est.predict(eval_input_fn))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzv0344pg/model.ckpt-140
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [9]:
print(stats_test['Player'].sort_index())
print("----------------------------------")
print(pos_test.sort_index())
print("----------------------------------")
print(pd.DataFrame(result[39]['probabilities']).head())
print(pos_vals)

20               D.J. Augustin\augusdj01
21               Deandre Ayton\aytonde01
27             Harrison Barnes\barneha02
35               Kent Bazemore\bazemke01
39                 Bradley Beal\bealbr01
40               Malik Beasley\beaslma01
46                  Jordan Bell\belljo01
51               Dāvis Bertāns\bertada01
53                  Khem Birch\birchkh01
57                Eric Bledsoe\bledser01
61                Jonah Bolden\boldejo01
75               Miles Bridges\bridgmi02
76              Oshae Brissett\brissos01
82                Jaylen Brown\brownja02
84              Sterling Brown\brownst02
90                  Alec Burks\burksal01
95               Bruno Caboclo\cabocbr01
96               Bruno Caboclo\cabocbr01
113            Wilson Chandler\chandwi01
114                Joe Chealey\chealjo01
119            Marquese Chriss\chrisma01
122                 Gary Clark\clarkga01
135                  Quinn Cook\cookqu01
141               Allen Crabbe\crabbal01
144             